## Using the API for extraction of structured data from palliative care consult questions

By: Kent McCann MD

Website: [Kent McCann MD](https://kmacman.github.io/)

# *Do not send any protected patient information to the publicly hosted API. It is strictly meant for demonstration purposes.*

In [2]:
#Importing dependencies
import requests
import json
import pandas as pd

C:\Users\kentm\AppData\Local\Temp\ipykernel_40336\1372607258.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
#This defines a class to submit a consult question to the API with methods for parsing out the json response:
class PalliAPI:
    def __init__(self, question):
        self.url = 'http://palliapi.kentmccannmd.com/analyze' #Change this to `http://localhost:80/analyze` if running the local Dockerized API.
        self.question = question
        payload = {'text': self.question}
        try:
            response = requests.post(self.url, data=json.dumps(payload))
            response.raise_for_status()  # Raises an HTTPError for bad responses
            self.data = response.json()
        except requests.RequestException as e:
            self.data = {}
            print(f"Failed to fetch data: {e}")

    def get_reason_1(self):
        return self.data.get('reason_code_1', "No data available")

    def get_reason_2(self):
        return self.data.get('reason_code_2', "No data available")

    def get_reason_3(self):
        return self.data.get('reason_code_3', "No data available")
    
    def get_diagnosis_text(self):
        return self.data.get('primary_diagnosis_text', "No data available")
    
    def get_diagnosis_code(self):
        return self.data.get('primary_diagnosis_code', "No data available")

    def get_full_response(self):
        return self.data if self.data else "No data available"

## Using the API to parse a single consult question:

In [6]:
consult_question = '35 year old gentleman, metastic lung cancer, pain, dyspnea. Requesting GOC.'
r = PalliAPI(consult_question)
print('Diagnosis:', r.get_diagnosis_text())
print('Diagnosis Code:', r.get_diagnosis_code())
print('')
print('Reason Codes')
print(r.get_reason_1())
print(r.get_reason_2())
print(r.get_reason_3())
print('')
print('Full JSON Response:')
print(r.get_full_response())

Diagnosis: cancer_solid
Diagnosis Code: 1

Reason Codes
1
2
{}

Full JSON Response:
{'reason_code_1': 1, 'reason_code_2': 2, 'reason_code_3': {}, 'reason_codes_key': '1: symptom, 2: goals, 3: support', 'primary_diagnosis_code': 1, 'primary_diagnosis_text': 'cancer_solid'}


## Using the API to process a dataframe of consult questions

The example dataset is synthetic. The consult questions were generated by Google Gemini and the names were generated with the Faker library. Any names that correspond to real people are strictly coincidental.

In [8]:
df = pd.read_csv('Example_Data\Synthetic Palliative Care Orders 10.csv')
df.head(10) #dataframe before processing


,Unnamed: 0,patient_name,consult_question
0,0,Heather Bell,Patient recently diagnosed with advanced COPD ...
1,1,Larry Wheeler,Patient with advanced COPD experiencing increa...
2,2,Tiffany Sutton,"Patient with ALS progressing rapidly, now quad..."
3,3,Mary Newman,"Patient with metastatic lung cancer, experienc..."
4,4,Jeremy Mcguire,Patient with metastatic lung cancer experienci...
5,5,Taylor Kennedy,"Patient with late-stage ALS, suffering from pa..."
6,6,Lisa Wilson,"Patient with advanced stage COPD, frequent hos..."
7,7,Caroline Barber,Patient with end-stage COPD experiencing incre...
8,8,Rhonda Farmer,"Patient with advanced stage 4 COPD, increasing..."
9,9,Nicholas Bartlett,Patient with advanced COPD now requiring home ...


In [9]:
#create columns for reason codes 1-3, as well a column for diagnosis text applying the above class methods to the 'consult_question' column
df['reason_code_1'] = df['consult_question'].apply(lambda x: PalliAPI(x).get_reason_1())
df['reason_code_2'] = df['consult_question'].apply(lambda x: PalliAPI(x).get_reason_2())
df['reason_code_3'] = df['consult_question'].apply(lambda x: PalliAPI(x).get_reason_3())
df['diagnosis_text'] = df['consult_question'].apply(lambda x: PalliAPI(x).get_diagnosis_text())

In [10]:
df.head(10) #dataframe after processing

,Unnamed: 0,patient_name,consult_question,reason_code_1,reason_code_2,reason_code_3,diagnosis_text
0,0,Heather Bell,Patient recently diagnosed with advanced COPD ...,1,2,3,pulmonary
1,1,Larry Wheeler,Patient with advanced COPD experiencing increa...,1,2,{},pulmonary
2,2,Tiffany Sutton,"Patient with ALS progressing rapidly, now quad...",2,3,{},neurology
3,3,Mary Newman,"Patient with metastatic lung cancer, experienc...",1,{},{},cancer_solid
4,4,Jeremy Mcguire,Patient with metastatic lung cancer experienci...,1,{},{},cancer_solid
5,5,Taylor Kennedy,"Patient with late-stage ALS, suffering from pa...",1,2,{},neurology
6,6,Lisa Wilson,"Patient with advanced stage COPD, frequent hos...",2,{},{},pulmonary
7,7,Caroline Barber,Patient with end-stage COPD experiencing incre...,1,2,{},pulmonary
8,8,Rhonda Farmer,"Patient with advanced stage 4 COPD, increasing...",2,{},{},pulmonary
9,9,Nicholas Bartlett,Patient with advanced COPD now requiring home ...,1,2,{},pulmonary
